<a href="https://colab.research.google.com/github/akitim/Ligand_activity_prediction_GlobalAI2022/blob/main/ligand_activity_prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!python -m pip install rdkit-pypi

     |████████████████████████████████| 20.6 MB 1.2 MB/s 


In [48]:
import matplotlib.pyplot as plt
import numpy as np
from rdkit import Chem
from rdkit.Chem.Draw import IPythonConsole
import pandas as pd

In [182]:
# mol = Chem.MolFromSmiles('CNNc1ccc(C(O)c2ccccc2)c(O)c1')
# mol = Chem.MolFromSmiles('Nc1nc(O)c2c(n1)NCC(CNc1ccc(C(=O)N[C@@H](CCC(=O)[O-])C(=O)[O-])cc1)N2C=O.[Ca+2]')
# im = Chem.Draw.MolToImage(mol)

# plt.figure(figsize = (10, 10))
# ax = plt.axes(frameon=True)
# ax.imshow(im)
# ax.axis('off');

## Загружаем данные:

In [233]:
train_path = 'https://raw.githubusercontent.com/akitim/Ligand_activity_prediction_GlobalAI2022/main/data/train.csv?token=GHSAT0AAAAAABQ5SI3K2YCVUTOJJOZECRR2YQE6QBA'
test_path = 'https://raw.githubusercontent.com/akitim/Ligand_activity_prediction_GlobalAI2022/main/data/test.csv?token=GHSAT0AAAAAABQ5SI3LWUXFQVX7PCGHR3D4YQE6Q2Q'

# train = pd.read_csv(train_path)
train = pd.read_csv('/content/train.csv')
# test = pd.read_csv(test_path)
test = pd.read_csv('/content/test.csv')

train = train.loc[:, ['Smiles', 'Active']]
train.columns = ['smiles', 'active']
train['active'] = train['active'].astype('int32')

test = test.loc[:, ['Smiles']]
train.sample(5)

,smiles,active
5226,COc1c2ccoc2cc2oc(=O)ccc12,0
3183,O=C(c1ccc(Cl)cc1)N(Cc1ccccc1)C1CCCCC1,0
1859,O=C(CCCN1CCC2(CC1)C(=O)NCN2c1ccccc1)c1ccc(F)cc1,0
2233,CC(CNC(=O)c1cccnc1)NC(=O)c1cccnc1,0
3684,O=C(/C=C/c1ccccc1)NC(NC(=S)Nc1ccc(Cl)cc1)C(Cl)...,0


In [93]:
active_ind = train.groupby('active').groups[1]
not_active_ind = train.groupby('active').groups[0]
print(f"Active: {active_ind.size}\n\
Not active:{not_active_ind.size}")

Active: 206
Not active:5351


## Визуализируем активные молекулы:

In [196]:
smiles_active = train.iloc[active_ind, :]['smiles'].values
# fig, axs = plt.subplots(25, 2, figsize=(10, 100))
# axs = axs.ravel()

# for i, smpl in enumerate(smiles_active[50:100]):
#     mol = Chem.MolFromSmiles(smpl)
#     im = Chem.Draw.MolToImage(mol)
#     print(f'{i}) {smpl}')
#     # plt.figure(figsize = (3, 3))
#     # axs(i) = plt.axes(frameon=True)
#     axs[i].imshow(im)
#     axs[i].axis('off');

## Модель 0 (baseline)

In [232]:
submission = test.copy()
submission['Active'] = (np.random.rand(test.shape[0]) <= 206 / 5351)
#(np.random.rand(test.shape[0]) <= 206 / 5351) #.astype('int32')
# test.loc[test['Active'], 'Active'].count() / test.shape[0]
submission.to_csv

,Smiles,Active
0,COC(=O)Nc1ccc(-c2nc(N3CCOCC3)c3cnn(C4CCN(Cc5cc...,False
1,O=Cc1cn(Cc2ccc(Cl)cc2)c2ccccc12,True
2,NC(CO)C(=O)NNCc1ccc(O)c(O)c1O,False
3,Nc1nc(=O)n([C@H]2C[C@H](O)[C@@H](CO)O2)cc1Cl,False
4,Nc1ccc(C(=O)Nc2cccc(-c3nc(N4CCOCC4)c4oc5ncccc5...,False
...,...,...
1609,NS(=O)(=O)c1cc2c(cc1Cl)NC(C1CC3C=CC1C3)NS2(=O)=O,False
1610,Cc1cccc(Nc2ccncc2S(=O)(=O)NC(=O)NC(C)C)c1,False
1611,CCCC(=O)O[C@]1(C(=O)CO)CC[C@H]2[C@@H]3CCC4=CC(...,False
1612,CN(C)c1cccc(Oc2cnc(Nc3cccc(O)c3)nc2)c1,False
